In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from sklearn.metrics import roc_auc_score
from collections import OrderedDict 

In [2]:
# Seed random number generation
torch.manual_seed(62)
np.random.seed(62)

# Neural Net Model
### Fake Deep - CMS 155
I will first import the data.

In [3]:
df_train = pd.read_csv("./datasets/jeh_train_label-enc_iter-imp_linear-norm.csv")
# Rather than do real imputation,I just replace nans with zeros
df_train = df_train.fillna(0)
df_train.head(5)

,id,LATITUDE,LONGITUDE,STATE,DISCOVERY_TIME,FIRE_SIZE,FIPS_NAME,FIPS_CODE,SOURCE_REPORTING_UNIT_NAME,DATE,LABEL
0,0,38.205000,-120.335000,0,130.0,0.10,215,0.0,157,0,1
1,1,33.813100,-85.104300,1,1115.0,1.17,82,143.0,71,0,4
2,2,32.201000,-82.498700,1,1600.0,0.07,130,209.0,71,0,2
3,3,32.509300,-81.708600,1,1215.0,4.40,19,31.0,71,0,4
4,4,33.663889,-116.171944,0,0.0,0.20,215,0.0,14,0,2


In [4]:
df_test = pd.read_csv("./datasets/jeh_test_label-enc_iter-imp_linear-norm.csv")
df_test = df_test.fillna(0)
df_test.head(5)

,id,LATITUDE,LONGITUDE,STATE,DISCOVERY_TIME,FIRE_SIZE,FIPS_NAME,FIPS_CODE,SOURCE_REPORTING_UNIT_NAME,DATE
0,285382,34.346944,-117.442222,0,1605.0,0.2,158,71.0,145,0
1,285383,34.020390,-116.179970,0,1545.0,0.1,218,0.0,69,0
2,285384,38.068611,-120.276667,0,1200.0,0.1,196,109.0,170,0
3,285385,32.499971,-83.742573,1,0.0,0.4,87,153.0,47,1
4,285386,32.924940,-114.992530,0,126.0,0.1,89,25.0,18,1


Here I separate the data randomly into training and testing sets, with a 75/25 split. 

In [5]:
D = len(df_train.columns[1:-1])
N_total = len(df_train)
N = int(0.75 * N_total)
N_test = N_total - N
X_predict = df_test.to_numpy(dtype = float)[:, 1:]
try:
    X_train = np.load("./jeh_checkpoints/x_train.npy")
    Y_train = np.load("./jeh_checkpoints/y_train.npy")
    X_test = np.load("./jeh_checkpoints/x_test.npy")
    Y_test = np.load("./jeh_checkpoints/y_test.npy")
except FileNotFoundError:
    test_indices = np.random.choice(list(range(N_total)), size=N_test, replace=False)
    X_train = np.zeros([N, D], dtype = float)
    Y_train = np.zeros(N, dtype = int)
    X_test = np.zeros([N_test, D], dtype = float)
    Y_test = np.zeros(N_test, dtype = int)
    j = 0
    k = 0
    for i in range(len(df_train)):
        if i not in test_indices:
            X_train[j, :] = df_train.iloc[i, 1:-1]
            Y_train[j] = df_train.iloc[i, -1]
            j += 1
        else:
            X_test[k, :] = df_train.iloc[i, 1:-1]
            Y_test[k] = df_train.iloc[i, -1]
            k += 1
    np.save("./jeh_checkpoints/x_train.npy", X_train)
    np.save("./jeh_checkpoints/y_train.npy", Y_train)
    np.save("./jeh_checkpoints/x_test.npy", X_test)
    np.save("./jeh_checkpoints/y_test.npy", Y_test)

Next I will apply a basic normalization to each column. I am careful to normalize everything using only the training data. 

In [6]:
for i in range(D):
    mu = np.mean(X_train[:, i])
    stddev = np.std(X_train[:, i])
    X_train[:, i] = (X_train[:, i] - mu ) / stddev
    X_test[:, i] = (X_test[:, i] - mu ) / stddev
    X_predict[:, i] = (X_predict[:, i] - mu ) / stddev

In [7]:
# We require Y_train and Y_test to be from 0-3, not 1-4
Y_train = Y_train - 1
Y_test = Y_test - 1

I actually need to onehot encode the labels to the data set. In effect, my neural net will have 4 output units and I want the labels to emulate this. 

In [8]:
C = len(np.unique(Y_train))
Y_train_oh = np.zeros([len(Y_train), C])
Y_test_oh = np.zeros([len(Y_test), C])
for i in range(len(Y_train)):
    y = Y_train[i] - 1
    Y_train_oh[i, y] = 1
for i in range(len(Y_test)):
    y = Y_test[i] - 1
    Y_test_oh[i, y] = 1

I need to write a dataset class in order to use this set with pytorch. This is totally barebones, but I don't need to worry about streaming the dataset off the hard drive to multiple cores, since I have the memory to just store the entire dataset on each core. 

In [9]:
class Dataset(torch.utils.data.Dataset):
    "Dataset object for pytorch."
    def __init__(self, X, Y):
        'Initialization'
        self.Y = Y.astype(float)
        self.X = X.astype(float)

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.Y)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Load data and get label
        x = self.X[index]
        y = self.Y[index]
        return x, y

Let's use this class to actually construct dataset objects. 

In [10]:
train_dataset = Dataset(X_train, Y_train_oh)
test_dataset = Dataset(X_test, Y_test_oh)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1024, shuffle=True) 

I will use my GPU to try and speed up the neural net. 

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')
# When you are on a CUDA machine, this should print a CUDA device:
print(device)

cuda


I'll take a first stab at the model architecture. 

I will import some helper functions that I wrote in problem set 4. 

In [15]:
def train_model():
    
    model.to(device)
    model.train()

    for epoch in range(12):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            # Erase accumulated gradients
            optimizer.zero_grad()
            
            # Forward pass
            output = model(data.float())

            # Calculate loss
            loss = loss_fn(output, target.float())

            # Backward pass
            loss.backward()

            # Weight update
            optimizer.step()

        # Track loss each epoch
        print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))

def get_train_err():
    model.eval()
    correct = 0
    train_error = 0
    train_loss = 0
    # Turning off automatic differentiation
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data.float())
            train_loss += loss_fn(output, target).item() * len(target) # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=False).cpu().numpy()  # Get the index of the max class score
            
            # Convert the target back from onehot encoding
            target = target.cpu().numpy()
            target = target[:, 1] + target[:, 2] * 2 + target[:, 3] * 3
            print(target)
            # Determine the accuracy of the classification
            correct += np.sum(pred==target)
            train_error += roc_auc_score(target, 
                                         output.cpu(), 
                                         multi_class='ovr') * len(target)
            
    train_loss /= len(train_loader.dataset)
    train_error /= len(train_loader.dataset)
    print("Average Training ROC AUC: %.3f"%train_error)
    print('Training set: Average loss: %.4f, Accuracy: %d/%d (%.4f)' %
          (train_loss, correct, len(train_loader.dataset),
           100. * correct / len(train_loader.dataset)))
    
def get_test_err():
    # Putting layers like Dropout into evaluation mode
    model.eval()

    test_loss = 0
    correct = 0
    test_error = 0

    # Turning off automatic differentiation
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data.float())
            test_loss += loss_fn(output, target).item() * len(target)  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=False).cpu().numpy()  # Get the index of the max class score
            
            # Convert the target back from onehot encoding
            target = target.cpu().numpy()
            target = target[:, 1] + target[:, 2] * 2 + target[:, 3] * 3
            
            # Determine the accuracy of the classification
            correct += np.sum(pred==target)
            test_error += roc_auc_score(target, 
                                        output.cpu(), 
                                        multi_class='ovr') * len(target)

    test_loss /= len(test_loader.dataset)
    test_error /= len(test_loader.dataset)
    print("Average Testing ROC AUC: %.3f"%test_error)
    print('Test set: Average loss: %.4f, Accuracy: %d/%d (%.4f)' %
          (test_loss, correct, len(test_loader.dataset),
           100. * correct / len(test_loader.dataset)))

I will now try constructing and testing an **even deeper** neural net. 

In [13]:
height = 5000
narrow = 100
drop = 0.05
depth = 4
ordict = OrderedDict()
ordict['start'] = nn.Linear(D, narrow)
ordict['expand1'] = nn.Linear(narrow, height)
ordict['expand2'] = nn.ReLU()
ordict['expand3'] = nn.Dropout(drop)

# Construct the bulk of the net
for i in range(depth):
    ordict['1-%i'%i] = nn.Linear(height, height)
    ordict['2-%i'%i] = nn.ReLU()
    ordict['3-%i'%i] = nn.Dropout(drop)
    
# Narrow the net and bring it down to the last few nodes
ordict['narrow1'] = nn.Linear(height, narrow)
ordict['narrow2'] = nn.ReLU()
ordict['narrow3'] = nn.Dropout(drop)
ordict['final-1'] = nn.Linear(narrow, C)
ordict['final-2'] = nn.Softmax(dim=1)

# Pack all the layers into the model
model = nn.Sequential(ordict)

optimizer = torch.optim.Adam(model.parameters(), lr=float(1e-4))
loss_fn = nn.SmoothL1Loss()
print(model)

Sequential(
  (start): Linear(in_features=9, out_features=100, bias=True)
  (expand1): Linear(in_features=100, out_features=5000, bias=True)
  (expand2): ReLU()
  (expand3): Dropout(p=0.05, inplace=False)
  (1-0): Linear(in_features=5000, out_features=5000, bias=True)
  (2-0): ReLU()
  (3-0): Dropout(p=0.05, inplace=False)
  (1-1): Linear(in_features=5000, out_features=5000, bias=True)
  (2-1): ReLU()
  (3-1): Dropout(p=0.05, inplace=False)
  (1-2): Linear(in_features=5000, out_features=5000, bias=True)
  (2-2): ReLU()
  (3-2): Dropout(p=0.05, inplace=False)
  (1-3): Linear(in_features=5000, out_features=5000, bias=True)
  (2-3): ReLU()
  (3-3): Dropout(p=0.05, inplace=False)
  (narrow1): Linear(in_features=5000, out_features=100, bias=True)
  (narrow2): ReLU()
  (narrow3): Dropout(p=0.05, inplace=False)
  (final-1): Linear(in_features=100, out_features=4, bias=True)
  (final-2): Softmax(dim=1)
)


Now I will train the model and record the error. 

In [16]:
# train_model()
get_train_err()
get_test_err()

[2. 0. 3. ... 0. 0. 0.]
[0. 0. 0. ... 2. 0. 0.]
[2. 0. 0. ... 0. 0. 3.]
[3. 0. 0. ... 2. 0. 1.]
[2. 3. 0. ... 0. 1. 2.]
[0. 0. 0. ... 0. 0. 2.]
[0. 0. 1. ... 0. 0. 1.]
[0. 0. 0. ... 3. 0. 0.]
[0. 2. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 3. 1. 0.]
[0. 2. 0. ... 0. 0. 1.]
[0. 0. 3. ... 2. 0. 2.]
[0. 2. 2. ... 1. 2. 0.]
[0. 0. 0. ... 0. 0. 2.]
[0. 3. 2. ... 0. 1. 2.]
[0. 0. 0. ... 0. 0. 3.]
[2. 0. 0. ... 3. 0. 1.]
[0. 1. 0. ... 2. 0. 0.]
[1. 3. 1. ... 2. 0. 0.]
[0. 2. 2. ... 0. 0. 2.]
[0. 2. 1. ... 2. 1. 0.]
[0. 0. 2. ... 0. 0. 2.]
[1. 0. 3. ... 1. 0. 0.]
[0. 0. 1. ... 1. 0. 1.]
[0. 0. 0. ... 0. 0. 0.]
[2. 0. 1. ... 0. 1. 2.]
[0. 3. 2. ... 0. 1. 2.]
[0. 0. 0. ... 1. 0. 1.]
[0. 1. 0. ... 3. 0. 3.]
[0. 0. 3. ... 0. 0. 0.]
[2. 0. 3. ... 0. 2. 3.]
[1. 2. 0. ... 0. 0. 0.]
[0. 0. 1. ... 0. 2. 0.]
[0. 0. 0. ... 2. 0. 0.]
[2. 0. 0. ... 0. 2. 1.]
[0. 0. 0. ... 0. 0. 0.]
[2. 0. 1. ... 2. 1. 0.]
[0. 0. 3. ... 0. 2. 2.]
[2. 2. 2. ... 1. 0. 0.]
[0. 2. 2. ... 0. 3. 1.]
[0. 3. 0. ... 0.

Let's predict the testing error like I will have to later. 

In [19]:
model.eval()
pred_data = np.zeros([len(X_test), 4])
with torch.no_grad():
    pred_data[:, :] = model(torch.tensor(X_test).to(device).float()).cpu()

roc_auc_score(Y_test, pred_data, multi_class='ovr')

0.41059932920716025

In [18]:
len(Y_test)

71346

In [ ]:
len(X_)

This model looks good. Let's train it on everything. 

In [17]:
Y_oh = np.zeros([N_total, C])
X = np.zeros([N_total, D])
Y_oh[:N, :] = Y_train_oh
Y_oh[N:, :] = Y_test_oh
X[:N, :] = X_train
X[N:, :] = X_test
train_dataset = Dataset(X, Y_oh)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1024, shuffle=True)

# Redefine the model
model = nn.Sequential(ordict)
optimizer = torch.optim.Adam(model.parameters(), lr=float(1e-4))
loss_fn = nn.SmoothL1Loss()

# Retrain the model on all avaliable data and get the training error. 
train_model()
get_train_err()

Train Epoch: 1  Loss: 0.0622
Train Epoch: 2  Loss: 0.0656
Train Epoch: 3  Loss: 0.0586
Train Epoch: 4  Loss: 0.0631
Train Epoch: 5  Loss: 0.0630
Train Epoch: 6  Loss: 0.0575
Train Epoch: 7  Loss: 0.0648
Train Epoch: 8  Loss: 0.0609
Train Epoch: 9  Loss: 0.0628
Train Epoch: 10  Loss: 0.0578
Train Epoch: 11  Loss: 0.0617
Train Epoch: 12  Loss: 0.0624
Average Training ROC AUC: 0.802
Training set: Average loss: 0.0599, Accuracy: 185233/285382 (64.9070)


Now that the model is well trained, I will predict the labels and submit to kaggle. 

In [23]:
model.eval()
pred_data = np.zeros([len(X_predict), 5])
with torch.no_grad():
    pred_data[:, 1:] = model(torch.tensor(X_predict).to(device).float()).cpu()
pred_data[:, 0] = df_test["id"].values
pred_df = pd.DataFrame(pred_data, columns = ["id", "P1", "P2", "P3", "P4"])

In [25]:
pred_df = pred_df.astype({'id': 'int'})
pred_df.to_csv("./FakeDeep_Attempt4_NeuralNet.csv", index=False)

In [22]:
len(X_predict[0])

9

In [24]:
pd.read_csv("./FakeDeep_Attempt4_NeuralNet.csv")

,id,P1,P2,P3,P4
0,285382,0.508899,0.160748,0.289178,0.041175
1,285383,0.322410,0.013904,0.094937,0.568750
2,285384,0.419784,0.233136,0.227026,0.120054
3,285385,0.287099,0.516503,0.118180,0.078218
4,285386,0.377663,0.224787,0.331835,0.065715
...,...,...,...,...,...
73030,358412,0.861071,0.081721,0.041671,0.015536
73031,358413,0.335623,0.069149,0.587107,0.008120
73032,358414,0.604119,0.100185,0.287817,0.007879
73033,358415,0.573586,0.106984,0.297592,0.021839


In [26]:
pd.read_csv("./FakeDeep_Attempt4_NeuralNet.csv")

,id,P1,P2,P3,P4
0,285382,0.508899,0.160748,0.289178,0.041175
1,285383,0.322410,0.013904,0.094937,0.568750
2,285384,0.419784,0.233136,0.227026,0.120054
3,285385,0.287099,0.516503,0.118180,0.078218
4,285386,0.377663,0.224787,0.331835,0.065715
...,...,...,...,...,...
73030,358412,0.861071,0.081721,0.041671,0.015536
73031,358413,0.335623,0.069149,0.587107,0.008120
73032,358414,0.604119,0.100185,0.287817,0.007879
73033,358415,0.573586,0.106984,0.297592,0.021839
